In [101]:
import pandas as pd
import numpy as np
import random
df = pd.read_csv("snacks.csv")
# if df[df.Snack == "MazMaz"]["Value"].item() > 8:
#     print("salam")
    
upBound = 4
downBound = 2
maximumWeight = 10
population = []

In [102]:
class KnapsackItem:
    def __init__(self, key_, weight_, value_):
        self.key = key_
        self.value = value_
        self.weight = weight_

In [113]:
numberOfSolutions = 1000
for i in range(0, numberOfSolutions):
    numberOfItems = random.randint(downBound, upBound)
    itemsKey = sorted(random.sample(np.arange(len(df)).tolist(), numberOfItems))
    solution = []
    for j in range(0, len(df)):
        if j in itemsKey:
            randomWeight = random.uniform(0, df.iloc[j, 1])
            value = (randomWeight / df.iloc[j, 1])*df.iloc[j, 2]
            solution.append(KnapsackItem(j, randomWeight, value))
        else:
            solution.append(KnapsackItem(j, 0, 0))   
    population.append(solution)


In [114]:
def getSolutionPoint(solution_):
    weightSum = 0
    valueSum = 0
    numberOfNonzeros = 0
    for item in solution_:
        weightSum += item.weight
        valueSum += item.value
        if item.weight != 0:
            numberOfNonzeros += 1
    if weightSum > maximumWeight:
        return  maximumWeight - weightSum
    if numberOfNonzeros < downBound:
        return numberOfNonzeros - downBound
    if  numberOfNonzeros > upBound:
        return upBound - numberOfNonzeros
    else:
        return valueSum

    

In [115]:
population = sorted(population, key = lambda x : getSolutionPoint(x), reverse=True)

def weighted_choice(items):
    weights = [1 / (i + 1) for i in range(len(items))]
    return random.choices(items, weights=weights)[0]

